In [ ]:
from pycromanager import Dataset
import napari
import matplotlib.pyplot as plt
import json
import pandas as pd
import h5py 
import numpy as np
import dask.array as da
import os
from tempfile import TemporaryDirectory

In [ ]:
# df = pd.read_csv(r'/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/Classification/DUSP1_D_Final_merged_spots_df_MG3_Abs4_Apr24.csv')
df = pd.read_csv(r'/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/DUSP1_D_spots_RF_kept_2025-05-02.csv')

lstDatasetPaths = df['NAS_location'].unique()

In [ ]:
df[df['time'] == 90]['unique_spot_id'].nunique()

In [ ]:
lstDatasetPaths = r"/Volumes/share/" + lstDatasetPaths

In [ ]:
import re
# Remove only the inner folder, keeping filename
cleaned = np.array([
    re.sub(r'/DUSP1_Dex_\d+min_20220224/', '/', path)
    for path in lstDatasetPaths
])

print(cleaned[:3])

In [ ]:
cleaned

In [ ]:
i = 1
with h5py.File(cleaned[i], 'r') as f:
    arr = f['raw_images']
    mImgs = da.from_array(arr)
    temp_dir = TemporaryDirectory()
    temp_file_path = os.path.join(temp_dir.name, "mImgs.npy")
    np.save(temp_file_path, mImgs.compute())  # Compute the dask array before saving
    print(f"Array saved to: {temp_file_path}")

In [ ]:
temp_file_path = r'/var/folders/s8/2yzy3p4x18g77qsnmsz0rmxh0000gn/T/tmpacsxg3lu/mImgs.npy'
# load the array from the temporary file
loaded_array = np.load(temp_file_path)
print(f"Loaded array shape: {loaded_array.shape}")

In [ ]:
mImgs = loaded_array
mImgs = np.squeeze(mImgs) 
print(mImgs.shape) 

In [ ]:
# 2) Pick one NAS_location (or loop over all)
locs = df['NAS_location'].unique()
i = 1  # change as needed
spot_results = df[df['NAS_location'] == locs[i]]

# 3) Extract coordinates
fov = spot_results['fov'].values
z   = spot_results['z_px'].values
y   = spot_results['y_px'].values
x   = spot_results['x_px'].values

# 4) Stack into an (N,4) array: (frame, z, y, x)
points = np.vstack([fov, z, y, x]).T

In [ ]:
points.shape

In [ ]:
s = 1
viewer = napari.Viewer()
viewer.add_image(mImgs, channel_axis=1, name=f'{locs[i]}', colormap='gray', scale=(5*s, 1*s, 1*s))

# Add the points to the napari viewer
viewer.add_points(
    points,
    size=2,  # Adjust the size of the points
    face_color='red',
    edge_color='red',
    ndim=4,  # Match the dimensionality of the image
    name='Spot Results',
    scale=(5,1,1)
)

napari.run()